In [ ]:
import os
import scipy
import utils

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
datadir = '../datasets/NNN/'
fnames = utils.fnames(datadir)
# utils.compute_noise_ceiling(data) # data should be (units, conditions, reps)
dat = pd.read_pickle(os.path.join(datadir, 'fr_data.pkl'))

In [ ]:
rois = dat['roi'].unique()
for roi in rois:
    if roi is not None:
        if 'Unknown' not in roi:
            print(roi) 
            
            roi_data = dat[(dat['roi']==roi)]
            

            period = 'pre'
            
            # the array is still ragged
            all_pre = list(roi_data[period])
            num_units = len(all_pre)
            num_images = len(all_pre[0])

            # maximum number of reps for a single image
            max_reps = max(
                len(arr) if hasattr(arr, "__len__") else 0
                for pre in all_pre
                for arr in pre)

            # pad with nan
            stacked = np.full((num_units, num_images, max_reps), np.nan, dtype=float)
            for unit_i, pre in enumerate(all_pre):
                for img in range(num_images):
                    arr = np.array(pre[img])
                    reps_here = len(arr)
                    if reps_here > 0:
                        stacked[unit_i, img, :reps_here] = arr
                        
            (noiseceiling, ncsnr, signalvar, noisevar) = utils.compute_noise_ceiling(stacked)
            
            fig,axes = plt.subplots(1,2)

            ax = axes[0]
            sns.violinplot(noiseceiling, ax=ax)
            ax.set_title(f'{roi} % noise ceiling ({period} period)')

            ax = axes[1]
            sns.violinplot(ncsnr, ax=ax)
            ax.set_ylim(top=4)
            ax.set_title('ncsnr')
            plt.show()


In [ ]:
roi = 'MF1_9_F' #'MF1_9_F'
roi_data = dat[(dat['roi']==roi)]

period = 'pre'

# the array is still ragged
all_pre = list(roi_data[period])
num_units = len(all_pre)
num_images = len(all_pre[0])

# maximum number of reps for a single image
max_reps = max(
    len(arr) if hasattr(arr, "__len__") else 0
    for pre in all_pre
    for arr in pre)

# pad with nan
stacked = np.full((num_units, num_images, max_reps), np.nan, dtype=float)
for unit_i, pre in enumerate(all_pre):
    for img in range(num_images):
        arr = np.array(pre[img])
        reps_here = len(arr)
        if reps_here > 0:
            stacked[unit_i, img, :reps_here] = arr
            
print(stacked.shape) # (units, conditions, trials)

In [ ]:
(noiseceiling, ncsnr, signalvar, noisevar) = utils.compute_noise_ceiling(stacked)

In [ ]:
fig,axes = plt.subplots(1,2)

ax = axes[0]
sns.violinplot(noiseceiling, ax=ax)
ax.set_title(f'% noise ceiling ({period} period)')

ax = axes[1]
sns.violinplot(ncsnr, ax=ax)
ax.set_ylim(top=4)
ax.set_title('ncsnr')